# Readability Assessment throughConvolutional Neural Network

Readability assessment is a well known problem in natural language processing field. Giving someone the suitable text for his level of comprehension (not so easy and not so hard) could maximize his understanding and enjoyment. In this notebook we are trying to assess the readability of a given text regardless of the text subject using recurrent neural network.

## Corpus
> OneStopEnglish corpus: A new corpus for automatic readability assessment and text simplification  
> Sowmya Vajjala and Ivana Lučić  
> 2018  
> Proceedings of the Thirteenth Workshop on Innovative Use of NLP for Building Educational Applications, pages 297–304. Association for Computational Linguistics.  
> [url](http://aclweb.org/anthology/W18-0535). [bib file](https://aclanthology.coli.uni-saarland.de/papers/W18-0535/w18-0535.bib)

Please cite the above paper if you use this corpus in your research.

[![DOI](https://zenodo.org/badge/128919409.svg)](https://zenodo.org/badge/latestdoi/128919409)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

Now let's dive into our corpus.

In [1]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from ipywidgets import interact
from tabulate import tabulate
from IPython.display import display, HTML

from utils.loading import load_glove_embeddings
from utils.one_stop_english import load_corpus, corpus_to_words, corpus_to_pos, detokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional,Conv1D,MaxPooling1D,GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l1, l2, l1_l2

# from tensorflow.keras.engine.input_layer import Input

import numpy as np

# glove = load_glove_embeddings()
corpus = load_corpus()
articles, tags = corpus_to_words(corpus)

Reading level|Avg. Num. Words|Std. Dev|Number of Articles
---|---|---|---
Elementary|533.17|103.79|189
Intermediate|676.59|117.15|189
Advanced|820.49|162.52|189



In [2]:
@interact
def show_articles(i=(0,188,1), words=(0,1000,1)):
    data = [
        ["Advanced",detokenize(articles[i][:words])], 
        ["Intermediate",detokenize(articles[i+2][:words])], 
        ["Elementary",detokenize(articles[i+1][:words])]
    ]
    headers = ['Reading Level', 'Example']
    display(HTML(tabulate(data,tablefmt='html', headers=headers)+"<style>th,td {font-size: 10px}</style>"))

interactive(children=(IntSlider(value=94, description='i', max=188), IntSlider(value=500, description='words',…

In [3]:
maxlen = 1000 # Cuts off reviews after 100 words
training_samples = 450
validation_samples = 117
max_words = 45
articles, tags = corpus_to_pos(corpus)
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(articles)
print(tokenizer.word_index)
print(len(tokenizer.word_index))


{'nn': 1, 'in': 2, 'dt': 3, 'jj': 4, 'nns': 5, 'nnp': 6, ',': 7, '.': 8, 'rb': 9, 'prp': 10, 'vb': 11, 'vbd': 12, 'cc': 13, 'vbz': 14, 'to': 15, 'vbp': 16, 'cd': 17, 'vbn': 18, 'vbg': 19, 'prp$': 20, 'md': 21, 'wdt': 22, 'wrb': 23, 'wp': 24, 'jjr': 25, 'rp': 26, ':': 27, 'jjs': 28, 'ex': 29, 'rbr': 30, 'nnps': 31, '-rrb-': 32, '-lrb-': 33, 'rbs': 34, 'pdt': 35, '$': 36, 'fw': 37, 'uh': 38, 'wp$': 39, 'sym': 40, "''": 41, 'ls': 42, 'pos': 43, '``': 44}
44


In [4]:
sequences = tokenizer.texts_to_sequences(articles)
# print(sequences)
print(np.shape(sequences))


(567,)


In [5]:

data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
print(data)
print(np.shape(data))

[[23 10 11 ...  0  0  0]
 [23 10 11 ...  0  0  0]
 [23 10 11 ...  0  0  0]
 ...
 [ 4  5 21 ...  0  0  0]
 [ 6  5 21 ...  0  0  0]
 [ 4  5 21 ...  0  0  0]]
(567, 1000)


In [6]:
print(np.shape(tags))
labels = to_categorical(tags)
print(np.shape(labels))

(567,)
(567, 3)


In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
print(indices[0])
print(np.shape(data), np.shape(labels), np.shape(data[0]))

x_train= data[:training_samples]
y_train= labels[:training_samples]
print(np.shape(x_train), np.shape(y_train))

x_val =data[training_samples:]
y_val =labels[training_samples:]
print(np.shape(x_val), np.shape(y_val))
from tensorflow.python import keras


124
(567, 1000) (567, 3) (1000,)
(450, 1000) (450, 3)
(117, 1000) (117, 3)


In [35]:
data = data.reshape(data.shape[0], data.shape[1], 1)
model = Sequential()
model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.1, l2=0.1)))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.1, l2=0.1)))
model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation='softmax'))
# model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='acc',patience=10, verbose=1, mode='auto')
callbacks = [keras.callbacks.TensorBoard(log_dir='my_log_dir',histogram_freq=1)]
model.compile(optimizer=RMSprop(),
loss='categorical_crossentropy',
metrics=['acc'])
history = model.fit(data, labels,
epochs=500,  
batch_size=50,
validation_split=0.2,
callbacks=callbacks)
model.save_weights('pre_trained_glove_model.h5')
model.summary()

Train on 453 samples, validate on 114 samples
Epoch 1/500
453/453 [==============================] - 1s 1ms/step - loss: 39.0098 - acc: 0.3002 - val_loss: 35.3870 - val_acc: 0.3070
Epoch 2/500
453/453 [==============================] - 0s 196us/step - loss: 33.2434 - acc: 0.3135 - val_loss: 31.0013 - val_acc: 0.3158
Epoch 3/500
453/453 [==============================] - 0s 198us/step - loss: 29.1144 - acc: 0.3201 - val_loss: 27.3831 - val_acc: 0.3596
Epoch 4/500
453/453 [==============================] - 0s 194us/step - loss: 25.4749 - acc: 0.3444 - val_loss: 23.7335 - val_acc: 0.2807
Epoch 5/500
453/453 [==============================] - 0s 192us/step - loss: 22.1404 - acc: 0.4018 - val_loss: 20.5505 - val_acc: 0.3070
Epoch 6/500
453/453 [==============================] - 0s 185us/step - loss: 19.1125 - acc: 0.3797 - val_loss: 18.3630 - val_acc: 0.3246
Epoch 7/500
453/453 [==============================] - 0s 193us/step - loss: 16.4896 - acc: 0.3974 - val_loss: 15.3936 - val_acc: 0.30

Epoch 61/500
453/453 [==============================] - 0s 272us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3617 - val_acc: 0.3158
Epoch 62/500
453/453 [==============================] - 0s 202us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3615 - val_acc: 0.3158
Epoch 63/500
453/453 [==============================] - 0s 185us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3613 - val_acc: 0.3158
Epoch 64/500
453/453 [==============================] - 0s 195us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3614 - val_acc: 0.3158
Epoch 65/500
453/453 [==============================] - 0s 191us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3618 - val_acc: 0.3158
Epoch 66/500
453/453 [==============================] - 0s 188us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3618 - val_acc: 0.3158
Epoch 67/500
453/453 [==============================] - 0s 187us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3619 - val_acc: 0.3158
Epoch 68/500
453/453 [==========================

453/453 [==============================] - 0s 173us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3619 - val_acc: 0.3596
Epoch 122/500
453/453 [==============================] - 0s 173us/step - loss: 1.3628 - acc: 0.3201 - val_loss: 1.3622 - val_acc: 0.3158
Epoch 123/500
453/453 [==============================] - 0s 171us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3623 - val_acc: 0.3596
Epoch 124/500
453/453 [==============================] - 0s 191us/step - loss: 1.3628 - acc: 0.3267 - val_loss: 1.3624 - val_acc: 0.3596
Epoch 125/500
453/453 [==============================] - 0s 183us/step - loss: 1.3628 - acc: 0.3024 - val_loss: 1.3625 - val_acc: 0.3158
Epoch 126/500
453/453 [==============================] - 0s 190us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3626 - val_acc: 0.3158
Epoch 127/500
453/453 [==============================] - 0s 175us/step - loss: 1.3628 - acc: 0.3267 - val_loss: 1.3630 - val_acc: 0.3158
Epoch 128/500
453/453 [==============================] 

Epoch 181/500
453/453 [==============================] - 0s 175us/step - loss: 1.3626 - acc: 0.3377 - val_loss: 1.3624 - val_acc: 0.3158
Epoch 182/500
453/453 [==============================] - 0s 175us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3627 - val_acc: 0.3158
Epoch 183/500
453/453 [==============================] - 0s 195us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3628 - val_acc: 0.3158
Epoch 184/500
453/453 [==============================] - 0s 389us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3630 - val_acc: 0.3158
Epoch 185/500
453/453 [==============================] - 0s 189us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3631 - val_acc: 0.3158
Epoch 186/500
453/453 [==============================] - 0s 191us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3630 - val_acc: 0.3158
Epoch 187/500
453/453 [==============================] - 0s 270us/step - loss: 1.3627 - acc: 0.3377 - val_loss: 1.3631 - val_acc: 0.3158
Epoch 188/500
453/453 [==================

Epoch 241/500
453/453 [==============================] - 0s 179us/step - loss: 1.3630 - acc: 0.3377 - val_loss: 1.3647 - val_acc: 0.3158
Epoch 242/500
453/453 [==============================] - 0s 198us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3647 - val_acc: 0.3158
Epoch 243/500
453/453 [==============================] - 0s 201us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3646 - val_acc: 0.3158
Epoch 244/500
453/453 [==============================] - 0s 182us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3647 - val_acc: 0.3158
Epoch 245/500
453/453 [==============================] - 0s 203us/step - loss: 1.3628 - acc: 0.3377 - val_loss: 1.3645 - val_acc: 0.3158
Epoch 246/500
453/453 [==============================] - 0s 192us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3644 - val_acc: 0.3158
Epoch 247/500
453/453 [==============================] - 0s 179us/step - loss: 1.3629 - acc: 0.3377 - val_loss: 1.3644 - val_acc: 0.3158
Epoch 248/500
453/453 [==================

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()




In [57]:
result = model.evaluate(data, labels)
print(x[1])

567/567 [==============================] - 0s 146us/step
0.5097001764982466
